In [3]:
import pandas as pd
import pickle
import os

In [1]:
# construct AADT for each road

In [2]:
# construct distribution visualization

In [127]:
# count the crashes on each road
crash_count = pd.read_csv("crash_file.csv",header=None)

# id of each segement
segment_id = pd.read_csv("road_file.csv",header=None)

segment_id = segment_id.rename(columns={0: "WISLR_LINKID", 1: "FROM", 2:'TO'})

segment_id["CRASH"]= crash_count[2]

# get the length of segment
road_length = pd.read_csv("link_file.csv",header=None)

segment_id["LENGTH"] = road_length[2]

crash_data = segment_id.copy()

crash_data["crashToLength"] = crash_data["CRASH"]/crash_data["LENGTH"]

In [128]:
#load rural or urban

functional_class = pd.read_csv("functional_class.csv")

crash_data["functional_class"] = functional_class["functional_class"]

In [129]:
crash_data

,WISLR_LINKID,FROM,TO,CRASH,LENGTH,crashToLength,functional_class
0,4818580,1661200,1660820,0,4066,0.0,Urban Other Connecting Link Principal Arterial
1,4819008,1661678,1661705,0,158,0.0,Urban Local
2,4819009,1661704,1661705,0,264,0.0,Urban Local
3,4819014,1661723,1661697,0,581,0.0,Urban Local
4,4819015,1661705,1661723,0,264,0.0,Urban Local
...,...,...,...,...,...,...,...
3127,5571520,1663210,1663276,0,1003,0.0,Urban Collector
3128,5571521,1661558,1661551,0,317,0.0,Urban Local
3129,5571522,1661551,1661558,0,317,0.0,Urban Local
3130,5571523,2016715,2005025,0,1108,0.0,Urban Collector


In [137]:
averageCrashToLength = {}
minCrashToLength = {}

for i in set(crash_data["functional_class"]):
    rate = crash_data[crash_data["functional_class"]==i]['CRASH'].sum()/ \
        crash_data[crash_data["functional_class"]==i]['LENGTH'].sum()
    averageCrashToLength[i] = rate
    
    minCrashToLength[i] = crash_data[(crash_data["functional_class"]==i)&(crash_data["crashToLength"])!=0]['crashToLength'].min()
    
    if crash_data[(crash_data["functional_class"]==i)&(crash_data["crashToLength"])!=0].shape[0] == 0:
        minCrashToLength[i] = 0

In [138]:
minCrashToLength

{'Urban Collector': 0.00028620492272467084,
 'Rural Local': 8.059316569954868e-05,
 'Rural Minor Collector': 0,
 'Urban Local': 0.0005094243504839531,
 'Urban Other Connecting Link Principal Arterial': 0.0005571030640668524,
 'Urban Minor Arterial': 0.0004928536224741252,
 'Rural Minor Arterial': 0.00019327406262079628,
 'Rural Major Collector': 0.0003713330857779428}

In [139]:
updateCtoL = []
for index, row in crash_data.iterrows():
    fClass = row["functional_class"] 
    
    if row["crashToLength"] == 0:
        updateCtoL.append(minCrashToLength[fClass])
    else:
        updateCtoL.append(row["crashToLength"])
        
crash_data["crashToLength"] = updateCtoL

In [140]:
set(crash_data["functional_class"])

{'Rural Local',
 'Rural Major Collector',
 'Rural Minor Arterial',
 'Rural Minor Collector',
 'Urban Collector',
 'Urban Local',
 'Urban Minor Arterial',
 'Urban Other Connecting Link Principal Arterial'}

In [141]:
AADT = {'Rural Local': 200,
        'Rural Major Collector': 1600,
        'Rural Minor Arterial' : 2400,
        'Rural Minor Collector' : 700,
        'Urban Collector' : 2300,
        'Urban Local' : 400,
        'Urban Minor Arterial' : 5300,
        'Urban Other Connecting Link Principal Arterial':11500
       }

In [142]:
# add AADT to dataset
add_fClass_AADT = []
for index, row in crash_data.iterrows():
    fClass = row["functional_class"] 
    add_fClass_AADT.append(AADT[fClass])
    
crash_data["AADT"] = add_fClass_AADT

In [143]:
crash_data

,WISLR_LINKID,FROM,TO,CRASH,LENGTH,crashToLength,functional_class,AADT
0,4818580,1661200,1660820,0,4066,0.000557,Urban Other Connecting Link Principal Arterial,11500
1,4819008,1661678,1661705,0,158,0.000509,Urban Local,400
2,4819009,1661704,1661705,0,264,0.000509,Urban Local,400
3,4819014,1661723,1661697,0,581,0.000509,Urban Local,400
4,4819015,1661705,1661723,0,264,0.000509,Urban Local,400
...,...,...,...,...,...,...,...,...
3127,5571520,1663210,1663276,0,1003,0.000286,Urban Collector,2300
3128,5571521,1661558,1661551,0,317,0.000509,Urban Local,400
3129,5571522,1661551,1661558,0,317,0.000509,Urban Local,400
3130,5571523,2016715,2005025,0,1108,0.000286,Urban Collector,2300


In [215]:
test = 1/(365*5/100000000) * 1/crash_data["AADT"] * crash_data["crashToLength"] 
test2 = 1/(365*5/100000000) * 1/crash_data["AADT"] * crash_data["crashToLength"] * crash_data["LENGTH"]

In [216]:
crash_data["test"] = test
crash_data["test2"] = test2

In [217]:
crash_data

,WISLR_LINKID,FROM,TO,CRASH,LENGTH,crashToLength,functional_class,AADT,test,test2
0,4818580,1661200,1660820,0,4066,0.000557,Urban Other Connecting Link Principal Arterial,11500,0.002654,10.793001
1,4819008,1661678,1661705,0,158,0.000509,Urban Local,400,0.069784,11.025897
2,4819009,1661704,1661705,0,264,0.000509,Urban Local,400,0.069784,18.423018
3,4819014,1661723,1661697,0,581,0.000509,Urban Local,400,0.069784,40.544596
4,4819015,1661705,1661723,0,264,0.000509,Urban Local,400,0.069784,18.423018
...,...,...,...,...,...,...,...,...,...,...
3127,5571520,1663210,1663276,0,1003,0.000286,Urban Collector,2300,0.006818,6.838917
3128,5571521,1661558,1661551,0,317,0.000509,Urban Local,400,0.069784,22.121578
3129,5571522,1661551,1661558,0,317,0.000509,Urban Local,400,0.069784,22.121578
3130,5571523,2016715,2005025,0,1108,0.000286,Urban Collector,2300,0.006818,7.554855


In [218]:
crash_cnt = crash_data[["FROM","TO","CRASH"]]
crash_risk = crash_data[["FROM","TO","test"]]
crash_risk_dist = crash_data[["FROM","TO","test2"]]

In [219]:
crash_cnt.to_csv("crash_count0320.csv")
crash_risk.to_csv("crash_risk0320.csv")
crash_risk_dist.to_csv("crash_risk_dist0320.csv")

In [224]:
#create crash file for turn network
network = pd.read_csv("turning_network_linkID_updated.csv",header=None)

In [225]:
risk_dic = {}
count_dic = {}
risk_dist_dic = {}
for index, row in crash_data.iterrows():
    risk_dic[str(row["WISLR_LINKID"])] = row["test"]
    count_dic[str(row["WISLR_LINKID"])] = row["CRASH"]
    risk_dist_dic[str(row["WISLR_LINKID"])] = row["test2"]

In [226]:
risk_col = []
count_col = []
risk_dist_col = []
temp = 0
for index, row in network.iterrows():
    if row[0] in risk_dic:
        temp += 1
        risk_col.append(risk_dic[row[0]])
        count_col.append(count_dic[row[0]])
        risk_dist_col.append(risk_dist_dic[row[0]])
    else:
        risk_col.append(0)
        count_col.append(0)
        risk_dist_col.append(0)

In [227]:
network2 = network.copy()
network3 = network.copy()

In [228]:
network["risk"] = risk_col

In [229]:
network2["count"] = count_col

In [234]:
network3["dRisk"] = risk_dist_col

In [231]:
network = network[[1,2,"risk"]]

In [232]:
network2 = network2[[1,2,"count"]]

In [235]:
network3 = network3[[1,2,"dRisk"]]

In [210]:
network.to_csv("crash_risk0320.csv",index=False,header=False)

In [211]:
network2.to_csv("crash_count0320.csv",index=False,header=False)

In [236]:
network3.to_csv("crash_drisk0320.csv",index=False,header=False)

In [238]:
crash_data[["WISLR_LINKID","LENGTH","CRASH","AADT"]]

,WISLR_LINKID,LENGTH,CRASH,AADT
0,4818580,4066,0,11500
1,4819008,158,0,400
2,4819009,264,0,400
3,4819014,581,0,400
4,4819015,264,0,400
...,...,...,...,...
3127,5571520,1003,0,2300
3128,5571521,317,0,400
3129,5571522,317,0,400
3130,5571523,1108,0,2300
